In [4]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

cm.update('livereveal', {
              'width': "90%",
              'height': "90%",
              'scroll': True,
})

import warnings
warnings.filterwarnings('ignore')

bShowInline = True  # Set = False for document generation
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
from ipywidgets import widgets, Layout, Box, GridspecLayout


##Basic mcq

def create_multipleChoice_widget(description, options, correct_answer, hint):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            s = '\x1b[5;30;41m' + "try again" + '\x1b[0m' +"\n"
        with feedback_out:
            feedback_out.clear_output()
            print(s)
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    hint_out = widgets.Output()
    
    def hint_selection(b):
        with hint_out:
            print(hint)
            
        with feedback_out:
            feedback_out.clear_output()
            print(hint)
    
    hintbutton = widgets.Button(description="hint")
    hintbutton.on_click(hint_selection)
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         widgets.HBox([hintbutton, check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 

test = create_multipleChoice_widget('1. Let a, b, c be real numbers. If a<b and b<c, then:',
                                   ['a>c','a<c','b<c','c<a'],
                                   'a<c',
                                   "[hint]:")

correst = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'C',"[hint]: The interferogram is a single realization of a random process")

phslks = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'D',"[hint]: Recall the relationship of phase difference to displacement")

thermcorr = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'B',"[hint]: Which would be intrinsically dark?")

corrsens = create_multipleChoice_widget('',
    ['A','B','C','D'],
    'A',"[hint]: Reference the sensitivities section of the InSAR notebook")


<img src="Figs/Banner.png" width="100%" />
<font face="Calibri">
<br>
<font size="6"> <b> Synthetic Aperture Radar Interferometry <b> </font>

<font size="4"> <b> Correlation Theory <font color='rgba(200,0,0,0.2)'> </font> </b> </font>

<br> <img src="Figs/NASALogo.png" width="250" align="right" /> <br> 
<font size="4"> <b> Paul A Rosen and Scott Hensley</b> 
<font size="3">  <br>
<font> <b>Date: </b> July 25, 2020 </font>
</font>


This notebook presents the basics of interferometric synthetic aperture radar correlation. The student will be exposed to:

1. Decorrelation Theory
2. Relationship between Correlation and Phase Noise
3. Sources of Decorrelation
4. Intuition for Correlation



**Table of Radar and Spacecraft Parameters and constants used for any computed examples in this notebook**

| Parameter | Symbol | Value | Comment |
| --- | --- | --- | --- | 
| Wavelength | $\lambda $ | 0.24 m | (L-band)  
| Antenna Length | $L_a$ | 10 m | |  
| Antenna Width | $W_a$ | 2 m | |  
| Off-nadir boresight angle | $\theta_l$ | 30$^\circ$ | |
| Azimuth squint of  boresight angle | $\theta_{sq}$ | 0$^\circ$ | |
| Spacecraft Velocity | $v_{sc}$ | 7,500 m/s | Assumed constant |
| Spacecraft Altitude | $h_{sc}$ | 750,000 m | Assumed constant | 
| Radar Range Bandwidth | $B_r$ | 10 MHz | |
| Radar Pulse Duration | $\tau_r$ | 20 $\mu$s | Determines average power |
| Nominal Pulse Rate | $f_p$ | 1600 Hz | Determines average power and ambiguity levels |
| Peak Power on Transmit | $P_T$ | 4,000 W | Determines SNR |
| Radar Noise Temperature | $T_r$ | 300 K | Determines SNR |
| Corner Reflector Dimension | $L_{cr}$ | 2.4 m | Determines SNR |
| Speed of light         | $c $  | 299792456 m/s | |  
| Boltzman constant      | $k$   | 1.38064852 $\times$ 10$^{-23}$ m$^2$ kg s$^{-2}$ K$^{-1}$ | -228.6 dB |  
| Gravitational Constant | $G$   | 6.672 $\times$ 10$^{-11}$ m$^3$ kg$^{−1}$ s$^{−2}$ | |  
| Earth's Mass           | $M_E$ | 5.9742 $\times$ 10$^{24}$ kg | |

In [6]:
import numpy as np
Lambda = 0.24
L_a = 10.
W_a = 2. 
theta_l = 30. * np.pi/180.
theta_sq = 0. * np.pi/180.
v_sc = 7500.
h_sc = 750000. 
B_r = 20.e6
tau_r = 10.e-6
f_p = 1600.
P_T = 4000.
T_r = 300.
L_cr =2.4
c = 299792456 
k = 1.38064852e-23
G = 6.672e-11
M_E = 5.9742e24

# Interferometric Correlation

## Coherent nature of SAR images

Pixels in a radar image are a complex phasor representation of the coherent backscatter from the resolution element on the ground.  Superimposed on this is the propagation phase delay to that pixel and back to the radar.<img src="Figs/sarspeckle.png" width="450" align="center" />

The propagation phase delay is given above.  The backscatter phase delay derives from the coherent sum of contributions from all elemental scatterers in the resolution element.  Each element has backscatter 

\begin{equation}
A_{\epsilon,i} e^{j\phi_{\epsilon,i}}
\end{equation}

Each of these likes a range distance $\rho_{\epsilon,i}$ from a plane perpendicular to the range direction.  These contribute an additional phases to the coherent sum:

\begin{equation}
A_b e^{j\phi_{b}}= \sum_i A_{\epsilon,i} e^{j\phi_{\epsilon,i}} e^{j 4\pi \rho_{\epsilon,i}/\lambda}
\end{equation}

Since each pixel has a unique arrangement of scatterers, the amplitude and phase from pixel to pixel appears random.  This is the characteristic speckled nature of SAR images.

## The "sameness" of images in an interferometric pair

While the amplitude and phase of neighboring pixels in an image appear randomly related, the same pixel in another image observed from nearly the same vantage point will have nearly the same complex phasor representation of the coherent backscatter from the corresponding pixel.  However, if the vantage point is distinct, the propagation phase delay will be different. <img src="Figs/intsarspeckle.png" width="450" align="center" />

In the figure above, we note that the backscatter amplitude and phase of a pixel in the two images is nearly the same if the angular difference in observation is small.  In this case the interferogram will have a phase that is proportional to the propagation phase difference - the range difference:

\begin{equation}
i (\rho,x) = s_1 \cdot  s_2^* = A_b e^{j(\phi_{b}- 4\pi\rho_1/\lambda)} \cdot A_b e^{-j(\phi_{b}- 4\pi\rho_2/\lambda)}= A_b^2 e^{-j 4\pi(\rho_1-\rho_2)/\lambda}
\end{equation}

In correlation theory, we study the limits of this assumption of sameness.

## Correlation Definition

For signals $s_1(\rho,x)$ and $s_2(\rho,x)$ observed at interferometer locations 1 and 2, the correlation $\gamma$ is defined as 

\begin{equation}
\gamma \equiv \frac{\left|\langle s_1 s_2^\star\rangle \right|}{\sqrt{\langle s_1 s_1^\star\rangle\langle s_2 s_2^\star\rangle}}
\end{equation}

where $\langle \rangle$ here represents the expected value of the signal. 


Deterministic signals or combinations of deterministic signals have perfect correlation because the expected value of the amplitude and phase is the amplitude and phase itself. <img src="Figs/corrdef.png" width="230" align="right" /> 

In this case:

\begin{equation}
\gamma = \frac{\left| | s_1 | | e^{j\phi_1} | | s_2 | | e^{-j\phi_2} | \right|}{ \sqrt{|s_1|^2  |s_2|^2}} = 1
\end{equation}



Signals with random components have imperfect correlation because the expected value of the product will not be the product of the expected values.  In the lower left figure, the shaded circle represents the range possible random values that the complex numbers $s_1$ or $s_2$ might have. <img src="Figs/corrdef2.png" width="230" align="left" /> For this case:

\begin{equation}
\gamma = \frac{\left|\langle | s_1 || s_2 | e^{j(\phi_1-\phi_2)} \rangle \right|}{ \sqrt{|s_1|^2  |s_2|^2}} \leq 1
\end{equation}
because the expected power of the signal and the power of the noise add in the denominator, while the noise tends to cancel in the expectation of the product in the numerator due to the phase term. 

## Correlation elements

Interferometric signals lose correlation (decorrelate) due to 

- Thermal noise in the radar system and processor noise 
- Viewing geometry effects, including baseline (cross and along track) and volume
- Changes of the surface over time 

The signal correlation in an interferogram can be related to the phase error, which in turn is related to the height or displacement accuracy and the ability to unwrap the phase.

Low correlation leads to lower accuracy and more difficulty in unwrapping.

The term "correlation" and "coherence" are often used synonymously, though correlation is typically used in reference to the sameness properties of data.  Coherence is typically associated with sameness properties of wave fields. 


## Expectation estimator and correlation estimates

In any given interferogram comprised of images $s_1$ and $s_2$, the images represent a single realization of the random processes involved.  For example, the thermal noise in the radar changes the signal randomly over time, but we only sample each pixel at a fixed time.

To properly compute expectation $\langle s_1 s_2^\star \rangle$, we would need many realizations of each image.  Since these are not available, we need to devise another estimator for interferometric phase and correlation in the presence of noise.

For an interferogram assumed to have homogeneous backscatter statistics and a constant phase difference in a region of interest, the maximum likelihood estimator approximates the expected value:

\begin{equation}
\langle s_1 s_2^\star \rangle \rightarrow \sum_{mn} s_1 (\rho_{mn},x_{mn}) s_2^\star (\rho_{mn},x_{mn})
\end{equation}
where the indices $mn$ are over the homogeneous area.  The maximum likelihood estimator of the phase is unbiased under these assumptions.  It is increasingly clear as researchers push to see smaller and smaller signals that this assumption is not always valid.

The correlation estimator then follows the same approach:
\begin{eqnarray}
\hat{\gamma} & = & \frac{\left|\langle s_1 s_2^\star\rangle \right|}{\sqrt{\langle s_1 s_1^\star\rangle\langle s_2 s_2^\star\rangle}} \\
& \rightarrow & \frac{\left|\sum_{mn} s_1 (\rho_{mn},x_{mn}) s_2^\star (\rho_{mn},x_{mn}) \right|}{\sqrt{\sum_{mn} s_1(\rho_{mn},x_{mn})  s_1^\star (\rho_{mn},x_{mn}) \sum_{mn} s_2 (\rho_{mn},x_{mn}) s_2^\star (\rho_{mn},x_{mn})}}
\end{eqnarray}

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Correlation Estimator <b> </font>
</div>

The correlation estimator based on area summation is a biased estimator. What is the value of the correlation estimate $\hat{\gamma}$ if the area summation is a single pixel
    
A. $\hat{\gamma} = 0$
    
B. $\hat{\gamma} =\gamma$
    
C. $\hat{\gamma} = 1$ 
    
D. $\hat{\gamma} = \gamma/A$, where $A$ is the area of the pixel in meters


In [7]:
correst

## Correlation estimator statistics

Since the correlation is biased, the estimated value of the correlation is not the actual correlation, and the estimate depends on the correlation itself and the area over which the estimate is calculated.  This is illustrated in the figure for correlation values $\gamma = 0$ and $\gamma = 1$ and for 10 or 40 pixels used in the summation ("looks"). <img src="Figs/corrhist.png" width="400" align="left" /> 

The maximum likelihood values at low correlation are well biased away from the true value.  It is easy to assume better correlation than exists in low correlation regions.

Note also that the larger the area, the less likely the homogeneity assumption is valid. 

## Correlation Example

The figure below shows an example of what correlation can look like in repeat pass InSAR images.  The figures depict correlation in two scenarios, the left is short time, long baseline interferogram, the right is a long time, short baseline interferogram.  The color indicates correlation, with blue being 0 correlation and green being a correlation of 1. Note that in each cases the water is a dark blue, indicating a correlation value of around 0.2.  But liquid water always has a true correlation of 0, since the surface is constantly changing with time.  This illustrates the bias associated with estimating the correlation described above. <img src="Figs/correxample.png" width="700" align="center" /> 

## Correlation and phase error

Under the assumption that the phase estimate is unbiased, it is possible to calculate the minimum variance of the that estimator, the Cramer-Rao bound.  For the number of samples used in the summation $N>4$, the phase standard deviation approaches the Cramer-Rao bound:
<img src="Figs/phsstd.png" width="400" align="left" /> 

\begin{equation}
\sigma_{\Delta\phi} = \frac{1}{\sqrt{2N}}\sqrt{\frac{1-\gamma^2}{\gamma^2}}
\end{equation}

Where $\sigma_{\Delta\phi}$ is the phase noise, $\gamma$ is the actual interferometric correlation and $N$ is the number of samples ("looks").

From the figure, calculated numerically, it can be seen that the formula does not hold for low number of samples.  Clearly, the greater the number of samples, the lower the phase error for a given $\gamma$.  Also, the greater the correlation, the lower the phase error for a given number samples. 


<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Phase noise <b> </font>
</div>

Using a repeat pass interferometer with a 24 cm wavelength, if the correlation of the interferogram was $\gamma=0.5$, how many samples would be needed to determine surface displacement to 3 mm precision? Assume the spatial baseline is 0.
    
A. $N \approx 10 $
    
B. $N \approx 8 $
    
C. $N \approx 1002 $ 
    
D. $N \approx 61 $

In [8]:
phslks

## Thermal noise decorrelation

Radar electronics will add thermally generated noise to the image observations.  We can model the noise signal as a component associated with the surface, $s_1$ and $s_2$, which for now we assume to be deterministic, and an additive uncorrelated noise term $n$.

\begin{equation}
s_{1,{\rm n}} = A_b e^{j\phi_b} e^{-j 4 \pi \rho_1/\lambda} + n_1 \qquad s_{2,{\rm n}} = A_b e^{j\phi_b} e^{-j 4 \pi \rho_2/\lambda} + n_2
\end{equation}

These expressions can be substituted into the correlation expression:

\begin{eqnarray}
\gamma & = & \frac{\left|\langle (s_1+n_1)(s_2+n_2)^\star\rangle \right|}{\sqrt{\langle (s_1+n_1)(s_1+n_1)^\star\rangle\langle (s_2+n_2) (s_2+n_2)^\star\rangle}} = \frac{\left|s_1 s_2 \right|}{\sqrt{ (s_1^2+n_1^2) (s_2^2+n_2^2) }} \\
& = & \frac{\sqrt{P_1} \sqrt{P_2}}{\sqrt{P_1+N_1}\sqrt{P_2+N_2}} = \frac{1}{\sqrt{1 + {\rm SNR}_1^{-1}}\sqrt{1 + {\rm SNR}_2^{-1}}}
\end{eqnarray}

where ${\rm SNR}_i = P_i/N_i$ is the Signal-to-noise ratio of image $s_i$.  If the images are equal in SNR, then this reduces to
\begin{equation}
\gamma_{\rm SNR} = \frac{1}{1 + {\rm SNR}^{-1}}
\end{equation}

The SNR in an image is scene dependent.  Surface elements that reflect back to the radar poorly have low SNR and therefore poor correlation.  Bright targets tend to have high correlation.

Decorrelation is simply $1-\gamma$.

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Thermal noise correlation <b> </font>
</div>

Which image characteristic would lead to low correlation due to thermal noise?

A. Layover
    
B. Shadow
    
C. Forests 
    
D. Close-range imaging

In [9]:
thermcorr

## Baseline decorrelation - the lack of "sameness" of images in an interferometric pair

Revisiting the sameness of images in the interferometric pair, our previous assumption that the differing vantage point is negligible needs to be examined more fully.  While the amplitude and phase of neighboring pixels in an image appear randomly related, the same pixel in another image observed from nearly the same vantage point will have nearly the same complex phasor representation of the coherent backscatter from the corresponding pixel. <img src="Figs/intsarspeckle2.png" width="450" align="center" />

However, it is not exactly the same, being impacted by the small differences in the coherent sum that forms $A_b \exp(j\phi_b)$:

\begin{equation}
i (\rho,x) = s_1 \cdot  s_2^* = A_{b1} e^{j(\phi_{b1}- 4\pi\rho_1/\lambda)} \cdot A_{b2} e^{-j(\phi_{b2}- 4\pi\rho_2/\lambda)}
\end{equation}



## Baseline correlation functional forms

To calculate the form of the decorrelation as the vantage difference increases requires an integration of assumed random scatterers over a resolution cell. The expression for correlation due to baseline effects is 

\begin{equation}
\gamma_{g_{f}} = 1 - \frac{b_{\perp}}{b_c} = 1 - \frac{p b_{\perp} \Delta \rho}{\lambda \rho \tan \theta} = 1 - \frac{p b \cos(\theta -\alpha) \Delta \rho}{\lambda \rho \tan \theta} 
\end{equation}

where $\gamma_{g_{f}}$ is the geometric correlation for a flat surface, $b_c$ is the critical baseline length, $b$ is the baseline length, $\theta$ is the look angle, $\alpha$ is the baseline orientation angle, i.e. the angle the baseline vector makes with respect to the local horizontal, $\Delta \rho$ is the range resolution, $\lambda$ is the wavelength, and $p=1$ for single antenna transmit systems and $p=2$ for ping-pong or repeat pass systems. The critical baseline is defined as the perpendicular baseline at which the correlation goes to zero. <img src="Figs/seasatcorr.png" width="350" align="right" />

\begin{equation}
b_c = \frac{\lambda \rho \tan(\theta - \psi_c)}{p \Delta \rho}
\label{bcrit}
\end{equation}

For a sloped surface, the baseline correlation is:


\begin{eqnarray}
\gamma_{g} &=& 1 - \frac{b_{\perp}}{b_c} = 1 - \frac{p b_{\perp} \Delta \rho}{\lambda \rho \tan( \theta - \psi_c)} \\
&=& \gamma_{g_f} \left [1 - \left(\frac{1-\gamma_{g_f}}{\gamma_{g_f}} \right)\left(\frac{\tan \theta}{\tan (\theta - \psi_c)}-1\right)\right]
\end{eqnarray}


## Rotational correlation

If we observe from a slightly different azimuth vantage point (rather than elevation as in baseline decorrelation), the surface can be decorrelated in a similar manner.  The larger the vantage point difference, the greater the decorrelation.  The same phenomenon can occur if the surface rotates for some reason (e.g. shear in an earthquake).  <img src="Figs/rotcorr.png" width="300" align="left" /> 

The simplest way to think about this is in terms of the azimuth antenna beam of the radar.  If the azimuth pointing of one radar difference by a full beamwidth, then the correlation of the images will be zero.  If they are aligned then the correlation will be 1.  This is related to the Doppler spectrum.  The spectra must overlap for there to be correlation. <img src="Figs/rotcorrsim.png" width="300" align="left" />  For shorter wavelength radars, the beamwidth of an antenna of given length will be narrower, and will therefore experience greater decorrelation than a longer wavelength radar of the same dimension.  



## The "pixel antenna" view of baseline decorrelation

Remembering the ideas from antenna theory that an antenna radiates from an aperture of dimension $L$ in an angle given by $\theta_{\rm beamwidth} \approx \lambda/L$.  The image pixel which reflect radar signal back toward the radar can similary be considered a small antenna with dimensions of the image resolution.  The figure to the left illustrates the general geometry. <img src="Figs/pixant.png" width="250" align="left" />  The image to the right examines the pixel as a radiating antenna.  The dimension of the antenna presented to the radar is related to the range resolution, $\Delta\rho$ as shown in the figure to the right.  We call this the range resolution projected onto the range, $\Delta \rho_\perp$.  The beamwidth subtended by the radiation pattern is then <img src="Figs/pixant2.png" width="250" align="right" />

$\Delta\theta = \frac{\lambda}{p\Delta\rho_\perp}$

The critical baseline can then be thought of as the distance at the orbit over which the observations must be made to have correlation.  

## Other aspects of baseline decorrelation

This heuristic conceptualization of baseline decorrelation helps in understanding other aspects of geometry-related decorrelation.  We can consider two cases.  One with a volume of vegetation as depicted in the figure.<img src="Figs/volcorr.png" width="500" align="center" />

Within the resolution cell, the projected area in which scatterers in the canopy contribute to that cell is much greater than for a flat surface.  Therefore the pixel antenna is much larger and the surface appears more decorrelated for a fixed interferometric baseline.

At the opposite extreme, we have a resolution cell with a single bright scatterer present. 
<img src="Figs/ptcorr.png" width="500" align="center" />

In this case, the pixel antenna has a broad radiation pattern, and the correlation is high for very larger baselines.  These are so-called "permanent scatterers" which remain correlated over space, and usually over time as well.

An expression for the volumetric correlation is:
\begin{equation}
\gamma_v = \textrm{sinc}  \left( \frac{k_z h_v}{2} \right) = \textrm{sinc} \left( \frac{2 \pi p}{\lambda} \frac{h_v b \cos(\theta - \alpha)}{2 \rho \sin \theta}  \right) 
\end{equation}

Where $\gamma_v$ is the volumetric correlation, $h_v$ is the thickness of the volume, and $k_z$ is the wavenumber in the vertical direction given by

\begin{equation}
k_z = \frac{\partial \phi}{\partial h}= \frac{2 \pi p}{\lambda} \frac{b \cos(\theta - \alpha)}{\rho \sin \theta}=\frac{2 \pi p b_{\perp}}{\lambda \rho \sin \theta}
\end{equation}

## Temporal correlation

Motion of scatterers in a resolution cell from one time to the next, as depicted on the left,  will cause the coherent backscatter to differ, leading to a temporal decorrelation effect.<img src="Figs/tempcorrpic.png" width="300" align="left" /> 

If we assume a Gaussian distributed of scatterer displacement - i.e. the position of a scatterer in the first image is displaced in the second image according to a Gaussian probability function, then the temporal correlation is given by:

\begin{equation}
\gamma_t=e^{-\frac{1}{2}\left(\frac{4 \pi}{\lambda}\right)^2 \left(\sigma_c^2 \sin^2 \theta + \sigma_h^2 \cos^2 \theta\right) }
\end{equation}

where $\gamma_t$ is the temporal correlation, $\lambda$ is the wavelength, and $\sigma_{c}$ is the standard deviation of scatterer displacement between observations in the cross-range direction, and $\sigma_{h}$ is the standard deviation of scatterer displacement between observations in the vertical direction. <img src="Figs/tempcor.png" width="300" align="left" />

Given the wavelength dependence, it is clear that longer wavelengths are less sensitive to random displacements as shown in the simulation.  However, the sensitivity of the long wavelength interferometer is also lower, so better correlation does not necessarily imply better performance.  

## Total Correlation

Now that we've studied all these sources of correlation noise, we can write the expression for the total correlation

\begin{equation}
\gamma= \gamma_{snr} \gamma_g \gamma_t \gamma_v
\end{equation}

Where $\gamma$ is the total correlation, $\gamma_{snr}$ is the SNR correlation, $\gamma_g$ is the geometric correlation (which may include rotations), $\gamma_t$ is the temporal correlation and $\gamma_v$ is the volumetric correlation.  For single-pass systems, $\gamma_t$ is not present. 

Which sources of decorrelation do you see in these AIRSAR repeat pass interferometric images?
<img src="Figs/airsarcorr.png" width="700" align="center" /> What can you say about the wavelength dependence?

<div class="alert alert-warning">
<font size="5"> <b> Test your understanding - Correlation Sensitivities<b> </font>
</div>

From the InSAR notebook, we learned that a repeat-pass interferometer with zero baseline has better displacement sensitivity (to mm) than a single-pass interferometer with non-zero baseline (to m).  Given that temporal decorrelation can be significant in repeat-pass measurements but is zero for single-pass systems, why would the sensitivity to deformation in repeat pass still be superior?

A. Single-pass systems measure deformation through topography differencing, which due to triangulation typically is only accurate to meter scale.  Even with temporal decorrelation, the repeat-pass measurement is accurate at the wavelength scale.
    
B. Single-pass measurements usually have much more geometric decorrelation due to the long baseline so tend to be much less accurate.
    
C. Topographic errors in single-pass systems can throw off differential phase estimates. 
    
D. Repeat-pass systems don't suffer from rotational decorrelation effects as much as single-pass systems.

In [10]:
corrsens

<div class="alert alert-danger">
<font size="5"> <b> Homework on Correlation <b> </font>
</div>
    
1. Compute the SNR correlation for SNR levels of -5 dB, 0 dB, 5 dB, 10 dB and 20 dB. What is the associated phase noise for these levels of correlation assuming the number of looks is 15?
2. Compute the critical baseline length for a system having an observation range of 330000 m, a wavelength of 0.056 m, a look angle of $40^\circ$, is operating in single antenna transmit mode, and has a range resolution of 13 m. Reduce the range resolution to 6 m and compute the critical baseline. What happens if you halve the wavelength?
3. It is a mildly windy day and the branches of trees are swaying in the wind with an crosstrack RMS motion of 1.5 cm. Assuming a look angle of $35^\circ$ compare the temporal decorrelation for a X-band (3 cm wavelength), C-band (5.6 cm wavelength), L-band (24 cm wavelength) and P-band (80 cm wavelength) radars.